#### Learning Objectives
By the end of this lesson, you will be able to..

- Create interaction terms.
- Recognize when a log transformation is appropriate.
- Use regularization to reduce overfitting.
- Distinguish between time-series and cross-sectional data.
- Describe time-series data in terms of trend, seasonality, and noise.
- Explain how train/test splits need to be performed differently for time-series data.

### Interaction Terms

# Advanced Topics in Regression

## Using Transformed Variables in Regression Models

Linear regression creates a model that is linear in the features that you pass into it.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

In [ ]:
mammals_path = Path('.', 'data', 'mammals.txt')
cols = ['brain','body']
mammals = pd.read_csv(mammals_path, sep='\t', names=cols, header=0)
mammals = mammals.loc[mammals.loc[:, 'body'] < 200, :].sort_values('body')
mammals.head()

In [ ]:
from sklearn.linear_model import LinearRegression

X = mammals.loc[:, ['body']]
y = mammals.loc[:, 'brain']

linreg = LinearRegression()
linreg.fit(X, y)
y_fit = linreg.predict(X)
plt.plot(X.values, y_fit)
plt.scatter(X.values, y);

But it can capture non-linear relationships with your original features if you give it non-linear transformations of those features.

In [ ]:
# Re-run the regression with an additional squared term
mammals.loc[:, 'body_squared'] = mammals.loc[:, 'body']**2

X = mammals.loc[:, ['body', 'body_squared']]
y = mammals.loc[:, 'brain']

linreg = LinearRegression()
linreg.fit(X, y)
y_fit = linreg.predict(X)

plt.plot(mammals.loc[:, 'body'].values, y_fit)
plt.scatter(mammals.loc[:, 'body'], y);

### Polynomial Terms

A polynomial function of x has the form $c_0 + c_1x + c_2x^2 + c_3x^3 + \ldots$.

If you give a linear regression model $x$, $x^2$, and $x^3$ as features, for instance, it will find the $\beta_0$, $\beta_1$, $\beta_2$, and $\beta_3$ that minimizes mean-squared error for using $\beta_0 + \beta_1x + \beta_2x^2 + \beta_3x^3$ to predict $y$.

It can always recover simple linear regression by setting the coefficients on the higher-order terms to 0, so adding these higher-order terms only increases the set of relationships that the model can capture.

**Exercise.**

- How does adding higher-order polynomial terms as inputs to a linear regression model affect its bias and variance?

Decreases (or at least does not increase) bias, increases variance.

**Every additional polynomial term gives your model an additional chance to change directions.**

In [ ]:
# first-order
x = np.linspace(-1, 1, 100)
plt.plot(x, x);

In [ ]:
# second-order
plt.plot(x, x**2);

In [ ]:
# third-order
x = np.linspace(-.75, 1.5, 100)
plt.plot(x, -.4*x-x**2+x**3);

In [ ]:
# fourth-order
x = np.linspace(-1, 1, 100)
plt.plot(x, -x**2+x**4);

In [ ]:
# Too many polynomial terms leads to overfitting
fig = sns.lmplot(x='body', y='brain', data=mammals, ci=None, order=8);

In [ ]:
# An (n-1)-order polynomial can always fit n data points perfectly.
# Definitely overfitting!
fig = sns.lmplot(x='body', y='brain', data=mammals, ci=None, order=50);
ax = fig.axes
ax[0,0].set_ylim(0, 200);

**Including multiple transformations of one variable complicates coefficient interpretation.**

In [ ]:
mammals.loc[:, 'body_squared'] = mammals.loc[:, 'body']**2

X = mammals.loc[:, ['body', 'body_squared']]
y = mammals.loc[:, 'brain']

linreg = LinearRegression()
linreg.fit(X, y)
y_fit = linreg.predict(X)

plt.plot(mammals.loc[:, 'body'].values, y_fit)
plt.scatter(mammals.loc[:, 'body'], y);

In [ ]:
print(linreg.intercept_)
print(linreg.coef_)

**Exercise.**

- Write down the fitted model we just created.

$brain = .132 + .108*body + .0021 * body^2$

- How would you normally interpret the coefficient on `body` in this model? Why doesn't that interpretation work in this case?

You would normally interpret the coefficient on a variable in a linear regression model as telling you how your model's predictions change with a one-unit increase in that variable, holding all other variables fixed. That interpretation doesn't work here because you can't have a one-unit increase in `body` while holding `body^2` fixed.

**sklearn has a "transformer" that generates polynomial terms**

In [ ]:
# sklearn transformers have the same interface as "estimators" (models)
# except that you fit them on features and use them to transform features,
# rather than fitting them on features and a target and using them to predict
# target values.
from sklearn.preprocessing import PolynomialFeatures

X = mammals.loc[:, ['body']]
pf = PolynomialFeatures(degree=3, include_bias=False)
pf.fit(X)
pf.transform(X)

**Exercise.**

Use the Boston housing data for the exercises below.

In [ ]:
from sklearn.datasets import load_boston

boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.DataFrame(boston.target, columns=['MEDV'])
boston = pd.concat([X, y], axis=1)
boston.head()

- Create a linear regression model for MEDV against DIS with no higher-order polynomial terms.

In [ ]:
X = boston.loc[:, ['DIS']]
y = boston.loc[:, 'MEDV']

lr_boston1 = LinearRegression()
lr_boston1.fit(X, y)

- Create a linear regression model for y against X polynomial terms up to and including degree seven.

In [ ]:
pf = PolynomialFeatures(degree=7, include_bias=False)
pf.fit(X)
X7 = pf.transform(X)

lr_boston7 = LinearRegression()
lr_boston7.fit(X7, y)

- Use 5-fold cross-validation to choose the polynomial order between 1 and 10 that gives the best results in terms of MSE on held-out data. *Hint*: use `sklearn.model_selection.cross_val_score`.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

kf = KFold(n_splits=5, shuffle=True)

for poly_degree in range(1, 11):
    pf = PolynomialFeatures(degree=poly_degree, include_bias=False)
    X_poly = pf.fit_transform(X)
    lr = LinearRegression()
    score = np.mean(-cross_val_score(lr, X_poly, y, cv=kf, scoring='neg_mean_squared_error'))
    print(poly_degree, score)

- **Bonus:** Create a model with $DIS$ and $DIS^{-1}$ as features and score it using 5-fold cross-validation.

In [ ]:
kf = KFold(n_splits=5, shuffle=True)

X_poly = pd.DataFrame(boston.loc[:, ['DIS']])
X_poly.loc[:, '1/DIS'] = X.loc[:, 'DIS']**(-1)
lr = LinearRegression()
np.mean(-cross_val_score(lr, X_poly, y, cv=kf, scoring='neg_mean_squared_error'))

- **Bonus:** Create line plots of your models' fitted values as a function of DIS and overlay them on scatterplots of MEDV against DIS.

In [ ]:
y_pred_boston1 = lr_boston1.predict(X)

plt.plot(X.values, y_pred_boston1, color='red')
plt.scatter(boston.loc[:, 'DIS'], boston.loc[:, 'MEDV'], alpha=.2)

In [ ]:
pf = PolynomialFeatures(degree=3)
X3 = pf.fit_transform(X)

lr_boston3 = LinearRegression()
lr_boston3.fit(X3, y)

y_pred_boston3 = lr_boston3.predict(X3)

sorted_boston = pd.DataFrame(boston.loc[:, 'DIS'])
sorted_boston.loc[:, 'pred3'] = y_pred_boston3
sorted_boston.sort_values('DIS', inplace=True)

plt.plot(sorted_boston.loc[:, 'DIS'], sorted_boston.loc[:, 'pred3'], color='red')
plt.scatter(boston.loc[:, 'DIS'], boston.loc[:, 'MEDV'], alpha=.2)

In [ ]:
y_pred_boston7 = lr_boston7.predict(X7)

sorted_boston = pd.DataFrame(boston.loc[:, 'DIS'])
sorted_boston.loc[:, 'pred7'] = y_pred_boston7
sorted_boston.sort_values('DIS', inplace=True)

plt.plot(sorted_boston.loc[:, 'DIS'], sorted_boston.loc[:, 'pred7'], color='red')
plt.scatter(boston.loc[:, 'DIS'], boston.loc[:, 'MEDV'], alpha=.2)

In [ ]:
X_neg = pd.DataFrame(boston.loc[:, ['DIS']])
X_neg.loc[:, '1/DIS'] = X.loc[:, 'DIS']**(-1)

lr_boston_neg = LinearRegression()
lr_boston_neg.fit(X_neg, y)

y_pred_boston_neg = lr_boston_neg.predict(X_neg)

sorted_boston = pd.DataFrame(boston.loc[:, 'DIS'])
sorted_boston.loc[:, 'pred_neg'] = y_pred_boston_neg
sorted_boston.sort_values('DIS', inplace=True)

plt.plot(sorted_boston.loc[:, 'DIS'], sorted_boston.loc[:, 'pred_neg'], color='red')
plt.scatter(boston.loc[:, 'DIS'], boston.loc[:, 'MEDV'], alpha=.2)

**Notes.**

- In statistics, it is extremely unusual to use more than a third-order polynomial.
- Higher-order polynomials are more common in machine learning, where the emphasis is on predictive accuracy rather than understanding.
- It would be unusual to use a polynomial term without including all lower-order polynomial terms.
- In addition to polynomial terms (with positive integer exponents), it can also be beneficial to include terms with negative exponents (e.g. $x^{-1}=1/x$) and/or fractional exponents (e.g. $x^{1/2}=\sqrt{X}$).

# Interaction Terms

Sometimes the significance of one feature depends on the value of another feature.

For instance, perhaps median housing prices increase as you get closer to a major employment center *unless crime is high around that area*.

We can model these kinds of "interaction effects" by including the *products* of the interacting variables as features in our models.

For example:

$$MEDV = \beta_0 + \beta_1 * DIS + \beta_2 * CRIM + \beta_{12} (DIS * CRIM)$$

In [ ]:
# Implement the model above


**Exercise.** Write down the fitted model we just created.

In [ ]:
# check descriptive stats


**Recall the usual interpretation of the coefficient on DIS:** how much the model's prediction for MEDV changes with a one-unit increase in DIS, all else being equal (i.e. for a particular value of CRIM).

**With interaction terms, interpreting the coefficients for a feature DIS requires specifying particular values for the interacting variables.**

For instance, if CRIM is fixed at its 25th percentile value of 0.082, we get

$MEDV = 22.62 + 0.48 * DIS + 0.467 * CRIM - 0.527 (DIS * CRIM)$

$MEDV = 22.62 + 0.48 * DIS + 0.467 * 0.082 - 0.527 (DIS * 0.082)$

$MEDV = 22.62 + 0.48 * DIS + 0.038 - 0.043 * DIS$

$MEDV = 22.658 + 0.437 * DIS$

So **at CRIM=.082**, the model's prediction for MEDV increases by .437 when DIS increases by one. It's better on average to be close to employment centers when crime is low.

The story is different when CRIM has its 75th percentile value of 3.64:

$MEDV = 22.62 + 0.48 * DIS + 0.467 * CRIM - 0.527 (DIS * CRIM)$

$MEDV = 22.62 + 0.48 * DIS + 0.467 * 3.64 - 0.527 (DIS * 3.64)$

$MEDV = 22.62 + 0.48 * DIS + 1.70 - 1.92 * DIS$

$MEDV = 24.32 -1.44 * DIS$

**At CRIM=3.64**, the model's prediction for MEDV *decreases* by 1.44 when DIS increases by one. It's better on average to be farther from employment centers when crime is high.

**Exercise.**

- How does adding interaction terms affect a model's bias and variance?

- Using 5-fold cross-validation, calculate the MSE for a model predicting MEDV from DIS and CRIM without an interaction term. *Hint*: use sklearn.model_selection.cross_val_score

- Using 5-fold cross-validation, calculate the MSE for a model predicting MEDV from DIS and CRIM with an interaction term. *Hint*: use sklearn.model_selection.cross_val_score

### Log Transformations

When your data is very skewed, try a log transformation.

In [ ]:
mammals = (
    pd.read_csv(mammals_path, sep='\t', names=cols, header=0)
    .dropna()
    .sort_values('body')
    .reset_index(drop=True)
)
mammals = mammals.loc[mammals.loc[:, 'body'] < 200, :]
mammals.hist();
mammals.plot(kind='scatter', x='body', y='brain');

In [ ]:
log_mammals = np.log(mammals)
log_mammals.hist()
log_mammals.plot(kind='scatter', x='body', y='brain')

Because we applied a log transformation to $y$ as well as $x$, we need to be careful about how we interpret the MSE values.

In [ ]:
# Train and score a linear model in the original space.
# This model isn't overfitting significantly, so let's not
# worry about a train/test split.
from sklearn import metrics

X = mammals.loc[:, ['body']]
y = mammals.loc[:, 'brain']
lr_mammals = LinearRegression()
lr_mammals.fit(X, y)
y_pred = lr_mammals.predict(X)
metrics.mean_squared_error(y, y_pred)

In [ ]:
# Train and score a linear model in the log-transformed space.
# This model isn't overfitting significantly, so let's not
# worry about a train/test split.
log_mammals = np.log(mammals)
X_log = log_mammals.loc[:, ['body']]
y_log = log_mammals.loc[:, 'brain']

lr_log_mammals = LinearRegression()
lr_log_mammals.fit(X_log, y_log)
y_pred_log = lr_log_mammals.predict(X_log)
metrics.mean_squared_error(y_log, y_pred_log)

Not a fair comparison! MSE for the second model is in log-space.

In [ ]:
# Get MSE for the log-log model in the original space
metrics.mean_squared_error(y, np.exp(y_pred_log))

**What's going on?**

In [ ]:
plt.scatter(X.values, y);
plt.plot(X, np.exp(y_pred_log));

In [ ]:
plt.scatter(X.values, y);
plt.plot(X, y_pred);

The model that we fit in log-log space is getting killed by the points in the top-right:

- MSE punishes large errors.
- Errors that are large in the original space don't look so large in log-log space, so the model doesn't focus on them as much as it "should."

The log-log model is still better in at least two respects:

- It conveys more understanding: modeling log of brain size as a linear function of log of body size plus random noise seems to capture what is really going on.
- It makes better predictions in terms of MAE.

In [ ]:
# Calculate MAE for the original and log-log models
print(metrics.mean_absolute_error(y, np.exp(y_pred_log)))
print(metrics.mean_absolute_error(y, y_pred))

**Notes**

- A log-transformed variable typically replaces the original variable in a regression analysis, unlike a polynomial term.
- You can apply a log transformation to any combination of your features and your target variable.

### Summary

- Linear regression *can* capture non-linear relationships *when you provide the appropriate non-linear transformations*.
- Every polynomial term you add allows your model to change directions once.
- Log transformations are awesome for skewed variables.

## Regularization

<a id="bonus-material-regularization"></a>
### Why Regularize?

Recall this overly complicated model:

In [ ]:
# Too many polynomial terms leads to overfitting
fig = sns.lmplot(x='body', y='brain', data=mammals, ci=None, order=8);

One way to make this model behave in a more reasonable way is to reduce the number of features we give it.

Another way is to *make it pay to use those features*. This approach is called **regularization**.

<a id="how-does-regularization-work"></a>
### How Does Regularization Work?

For a normal linear regression model, we estimate the coefficients using the least squares criterion, which minimizes the residual sum of squares (RSS).

For a regularized linear regression model, we minimize the sum of RSS and a "penalty term" that penalizes coefficient size.

**Ridge regression** (or "L2 regularization") minimizes: $$\text{RSS} + \alpha \sum_{j=1}^p \beta_j^2$$

**Lasso regression** (or "L1 regularization") minimizes: $$\text{RSS} + \alpha \sum_{j=1}^p |\beta_j|$$

- $p$ is the number of features.
- $\beta_j$ is a model coefficient.
- $\alpha$ is a tuning parameter:
    - A tiny $\alpha$ imposes no penalty on the coefficient size, and is equivalent to a normal linear regression model.
    - Increasing the $\alpha$ penalizes the coefficients and thus shrinks them.

<a id="lasso-and-ridge-path-diagrams"></a>
### Lasso and Ridge Path Diagrams

A larger alpha (toward the left of each diagram) results in more regularization:

- Lasso regression shrinks coefficients all the way to zero, thus removing them from the model.
- Ridge regression shrinks coefficients toward zero, but they essentially never reach zero.

Source code for the diagrams: [Lasso regression](http://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_lars.html) and [Ridge regression](http://scikit-learn.org/stable/auto_examples/linear_model/plot_ridge_path.html)

![Lasso and Ridge Coefficient Plots](./assets/lasso_ridge_path.png)

<a id="advice-for-applying-regularization"></a>
### Advice for Applying Regularization

**Features should be standardized** so that the penalty is not sensitive to the scale of the variables.

**How should you choose between lasso regression and ridge regression?**

- Lasso regression is preferred if we believe many features are irrelevant or if we prefer a sparse model.
- Ridge can work particularly well if there is a high degree of colinearity in your model.
- If model performance is your primary concern, it is best to try both.
- Elastic net regression is a combination of lasso regression and ridge regression.

<a id="ridge-regression"></a>
### Example

- [Ridge](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) documentation
- **alpha:** must be positive, increase for more regularization
- **normalize:** scales the features (without using StandardScaler)

In [ ]:
bikes_path = Path('.', 'data', 'bikeshare.csv')
bikes = pd.read_csv(bikes_path, index_col='datetime', parse_dates=True)
bikes_dummies = pd.get_dummies(bikes, columns=['season']).drop('season_1', axis=1)

In [ ]:
# Include dummy variables for season in the model.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Fit a Ridge Regression model with alpha=0 (equivalent to linear regression)


In [ ]:
# Coefficients for a non-regularized linear regression


Be careful in interpreting these coefficients, because they are related to the *normalized* versions of the input variables.

In [ ]:
# Try alpha=0.1.


In [ ]:
# Examine the coefficients.


Notice that the model tries to spread the coefficients more evenly because the squared term particularly punishes large deviations from zero.

In [ ]:
# Try Lasso with alpha=0.1


In [ ]:
# Examine the coefficients.


# Time Series Data

## What Is Time Series Data?

**Cross-Sectional Data**

<img src="./assets/cross_sectional_data.png" width=400></img>

Data is collected at a single point in time for each individual.

**Time-Series/Longitudinal Data**

<img src="./assets/time_series_data.png" width=400></img>

Data for the same variables is collected over time for the same individuals.

**Exercise.**

/poll "Which of the data sets that we have worked with so far contain time series data? (Select as many as apply.)" "bikes" "grad school admissions" "titanic" "ufo" "drinks" "boston housing" "nba" "mammals"

## Trends, Seasonality, and Noise

Google searches for "data science" show a strong increasing trend over the last five years:

![](./assets/data_science.png)

Google searches for "gingerbread house" show strong annual seasonality:

![](./assets/gingerbread_house.png)

Google searches for "iphone" show both trend and seasonality:

![](./assets/iphone.png)

**Exercise.**

Go to https://trends.google.com/trends/ and search for a term that interests you. Post it on Slack with a sentence or two about any trend or seasonality that it exhibits.

## Train/Test Split with Time Series Data

All training data come from before all test data -- your model should not be peeking into the future!

# Practice

Work on the practice notebooks from Unit 3 in pairs using the driver/navigator approach. One person (the "driver") writes the code (sharing his or her screen) while the other person (the "navigator") continually makes suggestions and reviews the code. The driver should talk about what he or she is doing, ask for input, and generally keep the navigator engaged.

We will switch driver/navigator roles when are time is halfway up.

Pick a notebook you haven't worked on yet if possible, especially when you are the driver. If you are the navigator for a notebook you have already worked on, don't pull out your code. Let your partner take the lead and give feedback on the direction they take.

# Projects

- Final Project Pt 3 due today
- Unit Project 3 (optional) due next Thurs.

# Questions?

```
=========================================
@channel
Exit Ticket: https://goo.gl/forms/OUw4gyTiRKMOTI3t2        

#feedback
=========================================
```